In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from feature_engine.imputation import AddMissingIndicator, MeanMedianImputer

In [2]:
df_listings = pd.read_csv("../data/2023dic/d_listings.csv")
#df_listings = df_listings[['id', 'neighborhood_overview', 'host_id', 'host_since', 'host_location', 'host_about',
#                           'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
#                           'host_listings_count', 'host_total_listings_count', 'host_verifications',
#                           'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed',
#                           'latitude', 'longitude', 'room_type',
#                           'accommodates', 'bathrooms_text', 'beds', 'price',
#                           'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30',
#                           'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'first_review',
#                           'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
#                           'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
#                           'review_scores_value', 'calculated_host_listings_count',
#                           'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
#                           'calculated_host_listings_count_shared_rooms', 'reviews_per_month']]

In [3]:
df_listings.drop(labels=["listing_url", "name", "scrape_id", "last_scraped", "source", "description", "picture_url", "host_url",
                         "host_name", "host_thumbnail_url", "host_picture_url", "host_neighbourhood", "neighbourhood",
                         "neighbourhood_group_cleansed", "property_type", "amenities", "minimum_minimum_nights",
                         "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", "minimum_nights_avg_ntm",
                         "maximum_nights_avg_ntm", "calendar_updated", "calendar_last_scraped", "number_of_reviews_ltm",
                         "number_of_reviews_l30d", "license", "instant_bookable"],
                 axis=1,
                 inplace=True)

In [4]:
df_listings.head()

,id,neighborhood_overview,host_id,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6623,Close by is the Frari Church (known as Tiziano...,15016,2009-04-27,"Venice, Italy","I'm usually happy and busy, often compelled to...",within a day,100%,74%,t,...,4.99,4.89,4.86,4.96,4.92,3,3,0,0,1.08
1,6624,We are in the middle of a residential area cal...,15016,2009-04-27,"Venice, Italy","I'm usually happy and busy, often compelled to...",within a day,100%,74%,t,...,4.97,4.93,4.87,4.95,4.93,3,3,0,0,0.59
2,12074,"There are plenty of bars, restaurants and pizz...",15016,2009-04-27,"Venice, Italy","I'm usually happy and busy, often compelled to...",within a day,100%,74%,t,...,4.98,4.90,4.88,4.88,4.89,3,3,0,0,1.21
3,27116,"The area is very beautiful and characteristic,...",116144,2010-04-30,"Venice, Italy",NaN,within an hour,100%,100%,f,...,4.90,4.78,4.81,4.94,4.78,3,1,2,0,2.55
4,44527,"Cannaregio is a well-connected, truly Venetian...",120215,2010-05-07,"Venice, Italy","I (Marc) have lived in Venice all my life, stu...",within an hour,100%,100%,t,...,4.76,4.97,4.93,4.89,4.67,2,2,0,0,0.83


## Feature engineering

- `first_review` to `last_review` as date span
- `host_listings_count` as a % of `host_total_listings_count`
- Manage `neighbourhoods_cleansed` as a OHE of most frequent categories
- Distance between host home and listing location
- Distance between listing and relevant locations in town
- `host_since` encoded as *days of activity until period end (end of dataset scraping)*
- Sentiment of `neighborhood_overview` (investigate best sentiment technique for descriptions of appartments)
- Sentiment of `host_about` (investigate best sentiment technique for description of people)
- `host_id` and `id` as categorial
- `host_response_time` as ordinal variable
- string manipulation for `host_response_rate` and `host_acceptance_rate`
- `host_is_superhost` as binary categorial
- `host_verifications` as encoded in previous script
- `host_has_profile_pic` as binary
- `host_identity_verified` as binary
- keep `room_type` instead of `property_type` and make `room_type` a categorial with OHE
- `accomodates` used with `baths`, `beds` to compute the rate of beds and bathrooms for every person
- `price` with string manipulation
- `minimum_nighs_avg_ntm` as float
- `maximum_nights_avg_ntm` as float
- `has_availability` as binary
- all the `has_availability_NUMBER` as a % of the NUMBER of the feature
- `number_of_reviews` as an integer
- `review_scores_rating` as float
- all the reviews scores as float
- remove `calculated_host_listings_count` and keep the other three BUT **set them as % of the total host listings**
- `reviews_per_month` as float
- `longitude` and `latitude` standardization (because the values are both negatives and positives)

## Transform feature datatypes

In this section we execute the feature engineering without dealing with null values.
We do it because once the types are cleaned, we want to plot a bit the data and explore it to see what is going on
with NAs, frequency distribution, numeric distributions etc.
In order to do so, we need to:
1. Generalise the pipeline, because we would like to apply this script also to other similar dataset
2. Return exceptions for NAs, to carry them on to the data exploration section

> ***NOTE*** that the `feature-engine` library enables us to split the dataset into train and test just after the data type and feature engineering. This because the library contains some functions for [preprocessing](https://feature-engine.trainindata.com/en/latest/user_guide/preprocessing/index.html) that can deal with removed rows and features afterwards

In [5]:
df_listings.dtypes

id                                                int64
neighborhood_overview                            object
host_id                                           int64
host_since                                       object
host_location                                    object
host_about                                       object
host_response_time                               object
host_response_rate                               object
host_acceptance_rate                             object
host_is_superhost                                object
host_listings_count                               int64
host_total_listings_count                         int64
host_verifications                               object
host_has_profile_pic                             object
host_identity_verified                           object
neighbourhood_cleansed                           object
latitude                                        float64
longitude                                       

In [ ]:
# https://feature-engine.trainindata.com/en/latest/quickstart/index.html